## 데이터 조작 / 고차 함수 (High Order Function)

이 노트북에서는 온라인 소매 판매 데이터로 작업하게 됩니다. `NULL` 값과 비표준 날짜 형식을 포함하는 데이터 열로 작업합니다.

다음 쿼리를 실행하여 Spark SQL에서 null 값 및 타임스탬프를 사용하고 관리하는 방법에 대해 알아봅니다. 다음을 수행합니다.

* 테이블 샘플
* 배열에서 개별 값에 액세스
* 월, 일 또는 연도와 같은 `DateType` 값의 일부에 액세스

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DataManipulation") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

spark

### table 생성

`OnlineRetail.csv`로 부터 `outdoorProductsRaw` Table을 생성합니다.

In [4]:
spark.sql("""
DROP TABLE IF EXISTS outdoorProductsRaw;
""")

spark.sql("""
CREATE TABLE outdoorProductsRaw USING csv OPTIONS (
  path '/content/OnlineRetail.csv',
  header "true"
)
""")

""


In [5]:
spark.sql("DESCRIBE outdoorProductsRaw")

col_name,data_type,comment
InvoiceNo,string,NULL
StockCode,string,NULL
Description,string,NULL
Quantity,string,NULL
InvoiceDate,string,NULL
UnitPrice,string,NULL
CustomerID,string,NULL
Country,string,NULL


In [6]:
spark.sql("SELECT * FROM outdoorProductsRaw").limit(5)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEA...,6,12/1/2010 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEART...,8,12/1/2010 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLA...,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE...,6,12/1/2010 8:26,3.39,17850,United Kingdom


### 테이블 샘플링
SQL 언어로 샘플을 무작위 검색하는 일반적인 방법 `RAND()` 함수와 `LIMIT` 키워드를 사용하는 것 이지만 Spark SQL에는 대신 사용할 수 있는 `TABLESAMPLE`  내장 함수가 포함되어 있습니다.

`TABLESAMPLE` 함수를 사용하면 여러 행 또는 데이터의 특정 비율을 반환할 수 있습니다.

다음 셀에서 주어진 데이터 비율에 액세스하는 데 사용할 수 있음을 보여줍니다. 그러나 테이블 표시는 1,000행으로 제한됩니다. 요청한 데이터의 백분율이 1,000개 이상의 행을 반환하는 경우 처음 1,000개만 표시됩니다. 테이블의 2%를 표시하는 샘플을 'InvoiceDate'로 정렬합니다.

In [7]:
# 일반적인 표준 SQL을 이용한 random sampling - 생성된 난수를 기준으로 데이터 정렬
spark.sql("SELECT * FROM outdoorProductsRaw ORDER BY RAND() LIMIT 5;")

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
547050,22334,DINOSAUR PARTY BA...,6,3/20/2011 12:02,1.65,17795,United Kingdom
543592,84836,ZINC METAL HEART ...,24,2/10/2011 11:57,1.25,17243,United Kingdom
566221,22703,PINK CAT BOWL,3,9/9/2011 16:13,1.63,NULL,United Kingdom
C553479,23174,REGENCY SUGAR BOW...,-2,5/17/2011 11:50,4.15,13098,United Kingdom
562417,84637,KITCHEN FLOWER PO...,1,8/4/2011 16:32,4.96,NULL,United Kingdom


In [8]:
# TABLESAMPLE 내장함수 사용 - sample 갯수 지정
spark.sql("SELECT * FROM outdoorProductsRaw TABLESAMPLE (5 ROWS);")

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEA...,6,12/1/2010 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEART...,8,12/1/2010 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLA...,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE...,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [9]:
#  TABLESAMPLE 내장함수 사용 - sample 비율 지정
spark.sql("SELECT * FROM outdoorProductsRaw TABLESAMPLE (2 PERCENT) ORDER BY  InvoiceDate;")

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
540564,22617,"mouldy, thrown away.",-2600,1/10/2011 10:36,0,NULL,United Kingdom
540566,22148,EASTER CRAFT 4 CH...,2,1/10/2011 10:58,1.95,17811,United Kingdom
540604,84879,ASSORTED COLOUR B...,16,1/10/2011 11:38,1.69,15326,United Kingdom
540604,21871,SAVE THE PLANET MUG,12,1/10/2011 11:38,1.25,15326,United Kingdom
540630,37500,TEA TIME TEAPOT I...,2,1/10/2011 11:59,9.95,17849,United Kingdom
540638,72038P,damages,-990,1/10/2011 12:14,0,NULL,United Kingdom
540639,21787,RAIN PONCHO RETRO...,24,1/10/2011 12:28,0.85,13107,United Kingdom
540639,21877,HOME SWEET HOME MUG,12,1/10/2011 12:28,1.25,13107,United Kingdom
540642,22977,DOLLY GIRL CHILDR...,24,1/10/2011 13:22,1.25,12681,France
540642,22423,REGENCY CAKESTAND...,1,1/10/2011 13:22,12.75,12681,France


### null value Check

이 셀을 실행하여 테이블의 `Description` 열에 있는 `NULL` 값의 수를 확인합니다.

In [10]:
# Spark SQL을 사용하여 outdoorProductsRaw 테이블에서 Description 컬럼에 NULL 값이 있는 행의 개수를 계산
spark.sql("SELECT COUNT(*) FROM outdoorProductsRaw WHERE Description IS NULL;")

count(1)
1454


### temporary view 생성

다음 셀은 temporary view인 `outdoorProducts`를 만들면서 Null 값을 가진 Description 을 '기타' 로 채워 넣습니다.

**COALESCE**  
`COALESCE`는 입력된 값들을 왼쪽에서부터 순서대로 확인하며 첫 번째로 Null이 아닌 값을 반환합니다. 모든 값이 Null인 경우, 최종 결과는 Null이 됩니다.  
따라서  `COALESCE(Description, "기타")`는  `Description` 열의 모든 `NULL` 값을 `"기타"`로 교체합니다.

In [11]:
# COALESCE 함수로 첫 번째 NULL이 아닌 값을 반환
spark.sql("SELECT COALESCE(NULL, NULL, 'A', 'B') AS result;")

result
A


In [12]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW outdoorProducts AS
SELECT
  InvoiceNo,
  StockCode,
  COALESCE(Description, "기타") AS Description,  --  Description 컬럼에 NULL 값이 있는 경우 "기타"로 대체
  Quantity,
  InvoiceDate
FROM outdoorProductsRaw;
""")

spark.sql("SELECT * FROM outdoorProducts WHERE Description = '기타' LIMIT 5;")

InvoiceNo,StockCode,Description,Quantity,InvoiceDate
536414,22139,기타,56,12/1/2010 11:52
536545,21134,기타,1,12/1/2010 14:32
536546,22145,기타,1,12/1/2010 14:33
536547,37509,기타,1,12/1/2010 14:33
536549,85226A,기타,1,12/1/2010 14:34


날짜에는 월 및 연도에 대한 표준 자릿수가 없습니다. 예를 들어 '12/1/11'에는 두 자리 월과 한 자리 일이 있고 '1/10/11'에는 한 자리 월과 두 자리 일이 있습니다. 문자열을 날짜로 변환하는 형식을 지정하는 것은 쉽지만 형식은 테이블 전체에서 일관되어야 합니다. 날짜의 모든 구성 요소를 분리하고 시간 값을 완전히 삭제하여 이 문제에 대한 수정을 시도할 것입니다.

**`SPLIT`**

이 명령은 지정된 컬럼의 문자열 값을 분할하고 **배열**을 반환합니다.  배열의 첫 번째 값은 월입니다. 첫 번째 값을 월로 가져오고 싶기 때문에 `SPLIT(InvoiceDate, "/")[0]`과 같이 값을 표시하고 **`month`** 열의 이름을 바꿉니다. 요일은 두 번째 값이고 인덱스는 1입니다.  

`split(InvoiceDate, /, -1)` : `/` - 구분자, `-1` - 개수 제한 없이 분리

In [13]:
# InvoiceDate 를 배열로 분리
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW outdoorProducts AS
SELECT
  InvoiceNo,
  StockCode,
  COALESCE(Description, "기타") AS Description,
  Quantity,
  SPLIT(InvoiceDate, "/")
FROM outdoorProductsRaw;
""")

spark.sql("SELECT * FROM outdoorProducts LIMIT 5;")

InvoiceNo,StockCode,Description,Quantity,"split(InvoiceDate, /, -1)"
536365,85123A,WHITE HANGING HEA...,6,"[12, 1, 2010 8:26]"
536365,71053,WHITE METAL LANTERN,6,"[12, 1, 2010 8:26]"
536365,84406B,CREAM CUPID HEART...,8,"[12, 1, 2010 8:26]"
536365,84029G,KNITTED UNION FLA...,6,"[12, 1, 2010 8:26]"
536365,84029E,RED WOOLLY HOTTIE...,6,"[12, 1, 2010 8:26]"


In [14]:
# invoiceDate를 월, 일, 연으로 분리
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW outdoorProducts AS
SELECT
  InvoiceNo,
  StockCode,
  COALESCE(Description, "Misc") AS Description,
  Quantity,
  SPLIT(InvoiceDate, "/")[0] month,
  SPLIT(InvoiceDate, "/")[1] day,
  SPLIT(InvoiceDate, "/")[2] year
FROM
  outdoorProductsRaw;
""")

spark.sql("SELECT * FROM outdoorProducts LIMIT 5;")

InvoiceNo,StockCode,Description,Quantity,month,day,year
536365,85123A,WHITE HANGING HEA...,6,12,1,2010 8:26
536365,71053,WHITE METAL LANTERN,6,12,1,2010 8:26
536365,84406B,CREAM CUPID HEART...,8,12,1,2010 8:26
536365,84029G,KNITTED UNION FLA...,6,12,1,2010 8:26
536365,84029E,RED WOOLLY HOTTIE...,6,12,1,2010 8:26


`InvoiceDate` 열은 날짜와 시간을 포함하는 문자열이므로, 날짜의 각 부분은 슬래시로 구분되지만 날짜와 시간 사이에는 공백만 있습니다. 공백 구분 기호에서 문자열을 분할하는 **nested** `SPLIT` 함수를 포함시켜 연도를 시간으로부터 분리합니다.


In [15]:
# invoiceDate를 월, 일, 연으로 구분 - 시간 분리
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW outdoorProducts AS
SELECT
  InvoiceNo,
  StockCode,
  COALESCE(Description, "Misc") AS Description,
  Quantity,
  SPLIT(InvoiceDate, "/")[0] month,
  SPLIT(InvoiceDate, "/")[1] day,
  SPLIT(SPLIT(InvoiceDate, "/")[2], " ")[0] year
FROM
  outdoorProductsRaw;
""")

spark.sql("SELECT * FROM outdoorProducts LIMIT 5;")

InvoiceNo,StockCode,Description,Quantity,month,day,year
536365,85123A,WHITE HANGING HEA...,6,12,1,2010
536365,71053,WHITE METAL LANTERN,6,12,1,2010
536365,84406B,CREAM CUPID HEART...,8,12,1,2010
536365,84029G,KNITTED UNION FLA...,6,12,1,2010
536365,84029E,RED WOOLLY HOTTIE...,6,12,1,2010


## Higher-order functions (고차 함수) - TEXT

Spark SQL의 고차 함수를 사용하면 복잡한 데이터 유형으로 직접 작업할 수 있습니다. 계층적 데이터(hierarchical data)로 작업할 때 레코드는 배열 또는 맵 유형 개체로 자주 저장됩니다.  

이 노트북에서는 고차 함수(`TRANSFORM`, `FILTER`, `EXISTS`)를 문자열 배열에 적용합니다.

### Filter

[Filter](https://spark.apache.org/docs/latest/api/sql/#filter)를 사용하면 배열의 값이 특정 조건을 충족하는지 여부에 따라 새 열을 만들 수 있습니다.

다음의 예는 InvoiceDate에서 특정 연도(2011)가 포함된 데이터를 필터링합니다.
이를 통해 연도 조건에 따라 데이터를 분리하고 가공할 수 있습니다.  

`FILTER(SPLIT(InvoiceDate, "/"), part -> part LIKE '%2011%') AS FilteredYear`

- `SPLIT(InvoiceDate, "/")` : 입력 배열 <br>
- `part` : 반복자 변수의 이름. 이 이름을 선택한 다음 람다 함수에서 사용. 배열을 반복하면서 각 값을 한 번에 하나씩 함수로 순환.<br>
- *`->` :  함수의 시작. <br>
- `part LIKE '%2011%` : 이것이 함수입니다. 각 값에 `"2011"` 이 **포함되어 있는지** 확인하고 참이면 새 열 `FilteredYear`로 필터링됩니다.

In [16]:
# outdoorProducts 뷰 생성: MONTH가 11인 데이터만 필터링
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW outdoorProducts AS
SELECT
  InvoiceNo,
  StockCode,
  COALESCE(Description, "기타") AS Description,
  Quantity,
  SPLIT(InvoiceDate, "/") AS InvoiceDateArray,
  -- FILTER를 사용하여 배열에서 YEAR 포함 여부 확인
  FILTER(SPLIT(InvoiceDate, "/"), part -> part LIKE '%2011%') AS FilteredYear
FROM outdoorProductsRaw
WHERE InvoiceDate LIKE '%2011%';
""")

# 필터링된 결과 출력
spark.sql("SELECT * FROM outdoorProducts").limit(5)

InvoiceNo,StockCode,Description,Quantity,InvoiceDateArray,FilteredYear
539993,22386,JUMBO BAG PINK PO...,10,"[1, 4, 2011 10:00]",[2011 10:00]
539993,21499,BLUE POLKADOT WRAP,25,"[1, 4, 2011 10:00]",[2011 10:00]
539993,21498,RED RETROSPOT WRAP,25,"[1, 4, 2011 10:00]",[2011 10:00]
539993,22379,RECYCLING BAG RET...,5,"[1, 4, 2011 10:00]",[2011 10:00]
539993,20718,RED RETROSPOT SHO...,10,"[1, 4, 2011 10:00]",[2011 10:00]


### SUB-QUERY:

SQL의 **하위 쿼리**는 쿼리 내의 쿼리입니다. 여러 단계로 작업을 수행하는 데 유용합니다.

내부 쿼리에서 데이터를 처리하고 필요한 값을 계산한 뒤, 외부 쿼리에서 추가 조건으로 필터링을 수행합니다.  

size(FilteredYear) > 0 : FilteredYear 배열의 크기가 0보다 큰 데이터만 반환합니다.
즉, "2011"을 포함한 행만 유지됩니다.

In [17]:
spark.sql("""
SELECT
  *
FROM
  (
    SELECT
      InvoiceNo,
      StockCode,
      COALESCE(Description, "기타") AS Description,
      Quantity,
      SPLIT(InvoiceDate, "/") AS InvoiceDateArray,
      -- FILTER를 사용하여 배열에서 "2011"을 포함한 값만 추출
      FILTER(SPLIT(InvoiceDate, "/"), part -> part LIKE '%2011%') AS FilteredYear
    FROM
      outdoorProductsRaw
  )
WHERE
  size(FilteredYear) > 0;
""")

InvoiceNo,StockCode,Description,Quantity,InvoiceDateArray,FilteredYear
539993,22386,JUMBO BAG PINK PO...,10,"[1, 4, 2011 10:00]",[2011 10:00]
539993,21499,BLUE POLKADOT WRAP,25,"[1, 4, 2011 10:00]",[2011 10:00]
539993,21498,RED RETROSPOT WRAP,25,"[1, 4, 2011 10:00]",[2011 10:00]
539993,22379,RECYCLING BAG RET...,5,"[1, 4, 2011 10:00]",[2011 10:00]
539993,20718,RED RETROSPOT SHO...,10,"[1, 4, 2011 10:00]",[2011 10:00]
539993,85099B,JUMBO BAG RED RET...,10,"[1, 4, 2011 10:00]",[2011 10:00]
539993,20682,RED RETROSPOT CHI...,6,"[1, 4, 2011 10:00]",[2011 10:00]
539993,22961,JAM MAKING SET PR...,12,"[1, 4, 2011 10:00]",[2011 10:00]
539993,22667,RECIPE BOX RETROS...,6,"[1, 4, 2011 10:00]",[2011 10:00]
539993,22898,CHILDRENS APRON A...,8,"[1, 4, 2011 10:00]",[2011 10:00]


### Exists

[Exists](https://spark.apache.org/docs/latest/api/sql/#exists)는 배열에 있는 하나 이상의 요소에 대해 조건이 참인지 여부를 테스트합니다.

함수를 더 잘 이해하기 위해 이 코드 줄을 분석해 보겠습니다.

`EXISTS(DescriptionArray, desc -> desc = '기타') Misc`

- DescriptionArray : 입력 배열의 이름 <br>
- desc : 반복자 변수의 이름. 이 이름을 선택한 다음 람다 함수에서 사용합니다. 배열을 반복하면서 각 값을 한 번에 하나씩 함수로 순환시킵니다.
- `->` :  함수의 시작을 나타냅니다. <br>
- desc = '기타' :  각 값이 `"기타"` 값과 **동일한지** 검사합니다. 새 열인 `Misc`에 플래그(true/false)가 지정됩니다.

In [18]:
# SQL 표현식을 문자열로 작성하고 이를 DataFrame API에서 사용할 수 있도록 하는 함수
from pyspark.sql.functions import expr

data = [
    ("1001", ["기타", "전자제품"], 10),
    ("1002", ["가구", "주방용품"], 5),
    ("1003", ["기타", "생활용품"], 20)
]

columns = ["InvoiceNo", "DescriptionArray", "Quantity"]
df = spark.createDataFrame(data, columns)

# EXISTS 함수 사용
# withColumn - 기존 컬럼을 수정하거나 새 컬럼을 추가
df = df.withColumn("hasMisc", expr("EXISTS(DescriptionArray, desc -> desc = '기타')"))
df

InvoiceNo,DescriptionArray,Quantity,hasMisc
1001,"[기타, 전자제품]",10,true
1002,"[가구, 주방용품]",5,false
1003,"[기타, 생활용품]",20,true


`outdoorProductsRaw` table 에서 "2011"을 포함한 행만 유지

In [19]:
spark.sql("""
SELECT
  *
FROM
  (
    SELECT
      InvoiceNo,
      StockCode,
      COALESCE(Description, "기타") AS Description,
      Quantity,
      SPLIT(InvoiceDate, "/") AS InvoiceDateArray,
      -- EXISTS를 사용하여 배열에서 "2011"을 포함한 요소가 있는지 확인
      EXISTS(SPLIT(InvoiceDate, "/"), part -> part LIKE '%2011%') AS ContainsYear2011
    FROM
      outdoorProductsRaw
  )
WHERE
  ContainsYear2011 = TRUE;
""")

InvoiceNo,StockCode,Description,Quantity,InvoiceDateArray,ContainsYear2011
539993,22386,JUMBO BAG PINK PO...,10,"[1, 4, 2011 10:00]",true
539993,21499,BLUE POLKADOT WRAP,25,"[1, 4, 2011 10:00]",true
539993,21498,RED RETROSPOT WRAP,25,"[1, 4, 2011 10:00]",true
539993,22379,RECYCLING BAG RET...,5,"[1, 4, 2011 10:00]",true
539993,20718,RED RETROSPOT SHO...,10,"[1, 4, 2011 10:00]",true
539993,85099B,JUMBO BAG RED RET...,10,"[1, 4, 2011 10:00]",true
539993,20682,RED RETROSPOT CHI...,6,"[1, 4, 2011 10:00]",true
539993,22961,JAM MAKING SET PR...,12,"[1, 4, 2011 10:00]",true
539993,22667,RECIPE BOX RETROS...,6,"[1, 4, 2011 10:00]",true
539993,22898,CHILDRENS APRON A...,8,"[1, 4, 2011 10:00]",true


### Transform

[Transform](https://spark.apache.org/docs/latest/api/sql/#transform)은 제공된 함수를 사용하여 배열의 모든 요소를 변환합니다. SQL의 기본 제공 함수는 셀 내의 단순한 단일 데이터 유형에서 작동하도록 설계되었기 때문에 배열 값을 처리할 수 없습니다. Transform은 배열의 각 요소에 기존 함수를 적용하려는 경우에 특히 유용할 수 있습니다. 이 경우에는 `categories` 열의 모든 이름을 소문자로 다시 작성하려고 합니다.

함수를 더 잘 이해하기 위해 이 코드 줄을 분석해 보겠습니다.

`TRANSFORM(categories, cat -> LOWER(cat)) lwrCategories`

- `categories` : 입력 배열의 이름 <br>
- `cat` : 반복자 변수의 이름.<br>
- `->` :  함수의 시작 <br>
- `LOWER(cat)` : 입력 배열의 각 값에 대해 내장 함수 `LOWER()`가 적용되어 단어를 소문자로 변환

In [20]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW testProducts AS
SELECT
  "1001" AS InvoiceNo,
  ARRAY("Electronics", "Home Appliance", "Toys") AS categories,
  10 AS Quantity
UNION ALL
SELECT
  "1002" AS InvoiceNo,
  ARRAY("Books", "Stationery", "Furniture") AS categories,
  5 AS Quantity
UNION ALL
SELECT
  "1003" AS InvoiceNo,
  ARRAY("Gardening", "Kitchenware", "Outdoor") AS categories,
  20 AS Quantity;
""")

spark.sql("SELECT * FROM testProducts")

InvoiceNo,categories,Quantity
1001,"[Electronics, Hom...",10
1002,"[Books, Stationer...",5
1003,"[Gardening, Kitch...",20


In [21]:
# categories 배열의 모든 요소를 소문자로 변환하여 새로운 column으로 저장
spark.sql("""
SELECT
  InvoiceNo,
  categories,
  TRANSFORM(categories, cat -> LOWER(cat)) AS lwrCategories
FROM
  testProducts;
""")

InvoiceNo,categories,lwrCategories
1001,"[Electronics, Hom...","[electronics, hom..."
1002,"[Books, Stationer...","[books, stationer..."
1003,"[Gardening, Kitch...","[gardening, kitch..."
